# 第10章 序列的修改、散切和切片

## 10.1 Vector类: 用户自定义的序列类型

使用组合模式来实现Vector类, 而不使用继承。向量的分量存储在浮点数数组中, 而且还将实现不可变扁平序列所需的方法

## 10.2 Vector类第一版: 与Vector2d兼容

In [1]:
import reprlib
reprlib.repr([1, 3, 4, 5, 6, 7, 10, 29])

'[1, 3, 4, 5, 6, 7, ...]'

In [2]:
from typing import Iterable
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self) -> str:
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other: Iterable) -> bool:
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [3]:
Vector([3.1, 4.2]), Vector((3, 4, 5)), Vector(range(10))

(Vector([3.1, 4.2]),
 Vector([3.0, 4.0, 5.0]),
 Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...]))

## 10.3 协议和鸭子类型

示例 10-3 示例 1-1 的代码, 为了方便, 再次给出

In [4]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [*[str(n) for n in range(2, 11)], *list('JQKA')]
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]

In [22]:
deck = FrenchDeck()
len(deck)

52

## 10.4 Vector第二版: 可切片的序列

### 10.4.1 切片原理

In [7]:
class MySeq:
    def __getitem__(self, pos):
        return pos

In [8]:
s = MySeq()
s[10:-2:0]

slice(10, -2, 0)

In [15]:
[dir(slice)[i:i+5] for i in range(0, len(dir(slice)), 5)]

[['__class__', '__delattr__', '__dir__', '__doc__', '__eq__'],
 ['__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__'],
 ['__hash__', '__init__', '__init_subclass__', '__le__', '__lt__'],
 ['__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__'],
 ['__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices'],
 ['start', 'step', 'stop']]

<mark /> 切片实现背后的原理: slice </mark>

In [21]:
slice(-3, 10).indices(5), slice(-2, 2, 2).indices(1)

((2, 5, 1), (0, 1, 2))

### 10.4.2 能处理切片的`__getitem__`方法

In [23]:
import numbers
class Vector2(Vector):
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = f'{cls.__name} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [29]:
vec = Vector2([2, 3, 4])
vec[1], vec[2], vec[:]

(3.0, 4.0, Vector([2.0, 3.0, 4.0]))

## 10.5 vector类的第三版: 动态存储属性

In [32]:
class Vector3(Vector2):
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = "{.__name!r} object has no attrinbute {!r}"
        raise AttributeError(msg.format(cls, name))

In [33]:
v = Vector3([2, 4, 6, 1, 0])
v.x, v.y, v.z , 

(2.0, 4.0, 6.0)

## 10.6 Vector第4版: 散列和快速等值测试

## 10.8 本章小结

<img src="./images/第10章总结.jpg" width="70%">